<a href="https://colab.research.google.com/github/PulyaPolya/Fashion_mnist/blob/main/fashion_32_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# All

In [ ]:
import numpy as np
import tensorflow as tf
import csv
import pandas as pd
import json
from sklearn.utils import shuffle
from keras.datasets import mnist
from keras.datasets import fashion_mnist
from scipy.ndimage.interpolation import shift
from tensorflow.keras import layers
import random
import matplotlib.pyplot as plt
from keras.callbacks import LambdaCallback


def edit_data(x_train, y_train, x_test, y_test):
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
    x_train = x_train / 255.0

    x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
    x_test = x_test / 255.0
    y_train = tf.one_hot(y_train.astype(np.int32), depth=10)
    y_test = tf.one_hot(y_test.astype(np.int32), depth=10)
    return x_train, y_train, x_test, y_test

<ipython-input-1-b9b5c68dfe3e>:9: DeprecationWarning: Please use `shift` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  from scipy.ndimage.interpolation import shift


In [ ]:
def shift_image(image, dx='NAN', dy='NAN'):
    if dx =='NAN' or dy =='NAN':
        dx, dy = random.choice([(1, 0), (-1, 0), (0, 1), (0, -1), (0, 0)])
    shifted_image = shift(image, [dy, dx], cval=0, mode="constant")
    return shifted_image
def shift_image_np(x_batch, dx ='nan', dy ='nan'):
    if dx == 'nan'and dy == 'nan':
        dx, dy = random.choice([(1,0), (-1,0), (0,1), (0,-1), (0,0)])
    x_shifted = np.pad(x_batch, ((abs(dx), 0), (abs(dy), 0)), mode='constant')
    x_shifted = np.roll(x_shifted, (dx, dy), axis=(0, 1))[abs(dx):, abs(dy):]
    return x_shifted

In [ ]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [ ]:
import tensorflow as tf
from keras.datasets import fashion_mnist
from keras.callbacks import ModelCheckpoint, TensorBoard
import time
from tensorflow.keras import layers
from keras.callbacks import LambdaCallback
import numpy as np

class Epoch_Tracker:
  def __init__(self):
     self.epoch = 0
     self.change = True
  def increase(self):
    self.epoch +=1
    self.change = True

epoch_track = Epoch_Tracker()

start = time.time()
x_train, y_train, x_test, y_test = edit_data(x_train, y_train, x_test, y_test)

size_data = x_train.shape[0]
batch_size = 32
num_classes = 10
epochs =15

NAME = "Bayes"

def random_invert_img(x):
  #print(epoch_track.epoch)
  if epoch_track.epoch >= epochs:
      return x
  x_temp = x.numpy()
  x_temp = x_temp.reshape(x_temp.shape[0], 28,28)
  x_shifted = []
  for image in x_temp:
      x_shifted.append(shift_image_np(image))
  x_shifted = np.array(x_shifted)

  x_result = x_shifted.reshape(x_temp.shape[0],28,28,1)
  return x_result
def random_invert():
  return layers.Lambda(lambda x: random_invert_img(x))

random_invert = random_invert()
class RandomInvert(layers.Layer):
  def __init__(self, **kwargs):
    super().__init__(**kwargs)

  def call(self, x):
    return random_invert_img(x)





# Run

In [ ]:
x_train.shape

(60000, 28, 28, 1)

# Evolution

In [ ]:
tf.random.set_seed(1234) #fashion 32 evo fold 1
input_shape = (28, 28, 1)
conv1 = 112
conv2 = 79
cov3 = 56
l_rate = 6
kernel1 = 3
kernel2=3
kernel3 = 13
drop1 = 3
drop2 = 5


optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
# optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/fashion32/evo/fold1/Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/fashion32/evo/fold1/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)

history = model.fit(x_train, y_train,  #fashion 32 evo fold 1
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50


1688/1688 [==============================] - ETA: 0s - loss: 0.4339 - acc: 0.8429
Epoch 1: val_acc improved from -inf to 0.88967, saving model to /content/drive/MyDrive/experiments/fashion32/evo/fold1/cnn_saved_model/weights-improvement--01-0.31.hdf5
1688/1688 [==============================] - 104s 54ms/step - loss: 0.4339 - acc: 0.8429 - val_loss: 0.3055 - val_acc: 0.8897
Epoch 2/50
1688/1688 [==============================] - ETA: 0s - loss: 0.2907 - acc: 0.8945
Epoch 2: val_acc improved from 0.88967 to 0.91100, saving model to /content/drive/MyDrive/experiments/fashion32/evo/fold1/cnn_saved_model/weights-improvement--02-0.25.hdf5
1688/1688 [==============================] - 93s 55ms/step - loss: 0.2907 - acc: 0.8945 - val_loss: 0.2477 - val_acc: 0.9110
Epoch 3/50
1687/1688 [============================>.] - ETA: 0s - loss: 0.2550 - acc: 0.9084
Epoch 3: val_acc improved from 0.91100 to 0.91667, saving model to /content/drive/MyDrive/experiments/fashion32/evo/fold1/cnn_saved_model/we

In [ ]:
tf.random.set_seed(1234) #fashion 32 evo fold 2
input_shape = (28, 28, 1)
conv1 = 122
conv2 = 93
cov3 =63
l_rate = 5
kernel1 = 3
kernel2=5
kernel3 = 7
drop1 = 3
drop2 = 6


optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
# optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/fashion32/evo/fold2/Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/fashion32/evo/fold2/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)

history = model.fit(x_train, y_train,  #fashion 32 evo fold 2
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
1688/1688 [==============================] - ETA: 0s - loss: 0.4249 - acc: 0.8472
Epoch 1: val_acc improved from -inf to 0.89150, saving model to /content/drive/MyDrive/experiments/fashion32/evo/fold2/cnn_saved_model/weights-improvement--01-0.30.hdf5
1688/1688 [==============================] - 80s 46ms/step - loss: 0.4249 - acc: 0.8472 - val_loss: 0.2965 - val_acc: 0.8915
Epoch 2/50
1688/1688 [==============================] - ETA: 0s - loss: 0.2844 - acc: 0.8974
Epoch 2: val_acc improved from 0.89150 to 0.91133, saving model to /content/drive/MyDrive/experiments/fashion32/evo/fold2/cnn_saved_model/weights-improvement--02-0.25.hdf5
1688/1688 [==============================] - 74s 44ms/step - loss: 0.2844 - acc: 0.8974 - val_loss: 0.2455 - val_acc: 0.9113
Epoch 3/50
1687/1688 [============================>.] - ETA: 0s - loss: 0.2447 - acc: 0.9107
Epoch 3: val_acc improved from 0.91133 to 0.91833, saving model to /content/drive/MyDrive/experiments/fashion32/evo/fold2/cnn_save

In [ ]:
tf.random.set_seed(1234) #fashion 32 evo fold 3
input_shape = (28, 28, 1)
conv1 = 108
conv2 = 74
cov3 =61
l_rate = 6
kernel1 = 3
kernel2=7
kernel3 = 15
drop1 = 5
drop2 = 4


# optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/fashion32/evo/fold3/Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/fashion32/evo/fold3/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)

history = model.fit(x_train, y_train,  #fashion 32 evo fold 3
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
1688/1688 [==============================] - ETA: 0s - loss: 0.4207 - acc: 0.8483
Epoch 1: val_acc improved from -inf to 0.89733, saving model to /content/drive/MyDrive/experiments/fashion32/evo/fold3/cnn_saved_model/weights-improvement--01-0.29.hdf5
1688/1688 [==============================] - 74s 42ms/step - loss: 0.4207 - acc: 0.8483 - val_loss: 0.2863 - val_acc: 0.8973
Epoch 2/50
1687/1688 [============================>.] - ETA: 0s - loss: 0.2796 - acc: 0.8978
Epoch 2: val_acc improved from 0.89733 to 0.90950, saving model to /content/drive/MyDrive/experiments/fashion32/evo/fold3/cnn_saved_model/weights-improvement--02-0.25.hdf5
1688/1688 [==============================] - 71s 42ms/step - loss: 0.2797 - acc: 0.8977 - val_loss: 0.2457 - val_acc: 0.9095
Epoch 3/50
1687/1688 [============================>.] - ETA: 0s - loss: 0.2413 - acc: 0.9110
Epoch 3: val_acc improved from 0.90950 to 0.91933, saving model to /content/drive/MyDrive/experiments/fashion32/evo/fold3/cnn_save

In [ ]:
tf.random.set_seed(1234) #fashion 32 evo fold 4
input_shape = (28, 28, 1)
conv1 = 126
conv2 = 95
cov3 =40
l_rate = 7
kernel1 = 5
kernel2=5
kernel3 = 13
drop1 = 4
drop2 = 3


# optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/fashion32/evo/fold4/Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/fashion32/evo/fold4/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)

history = model.fit(x_train, y_train,  #fashion 32 evo fold 4
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
1688/1688 [==============================] - ETA: 0s - loss: 0.4014 - acc: 0.8537
Epoch 1: val_acc improved from -inf to 0.89117, saving model to /content/drive/MyDrive/experiments/fashion32/evo/fold4/cnn_saved_model/weights-improvement--01-0.29.hdf5
1688/1688 [==============================] - 76s 43ms/step - loss: 0.4014 - acc: 0.8537 - val_loss: 0.2917 - val_acc: 0.8912
Epoch 2/50
1687/1688 [============================>.] - ETA: 0s - loss: 0.2675 - acc: 0.9014
Epoch 2: val_acc improved from 0.89117 to 0.90967, saving model to /content/drive/MyDrive/experiments/fashion32/evo/fold4/cnn_saved_model/weights-improvement--02-0.25.hdf5
1688/1688 [==============================] - 72s 43ms/step - loss: 0.2675 - acc: 0.9014 - val_loss: 0.2473 - val_acc: 0.9097
Epoch 3/50
1688/1688 [==============================] - ETA: 0s - loss: 0.2304 - acc: 0.9148
Epoch 3: val_acc improved from 0.90967 to 0.91617, saving model to /content/drive/MyDrive/experiments/fashion32/evo/fold4/cnn_save

In [ ]:
tf.random.set_seed(1234) #fashion 32 evo fold 5
input_shape = (28, 28, 1)
conv1 = 45
conv2 = 72
cov3 =58
l_rate = 5
kernel1 = 3
kernel2=7
kernel3 = 9
drop1 = 4
drop2 = 5


optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
# optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/fashion32/evo/fold5/Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/fashion32/evo/fold5/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)

history = model.fit(x_train, y_train,  #fashion 32 evo fold 5
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50


1688/1688 [==============================] - ETA: 0s - loss: 0.4601 - acc: 0.8334
Epoch 1: val_acc improved from -inf to 0.87983, saving model to /content/drive/MyDrive/experiments/fashion32/evo/fold5/cnn_saved_model/weights-improvement--01-0.32.hdf5
1688/1688 [==============================] - 88s 45ms/step - loss: 0.4601 - acc: 0.8334 - val_loss: 0.3235 - val_acc: 0.8798
Epoch 2/50
1688/1688 [==============================] - ETA: 0s - loss: 0.3063 - acc: 0.8887
Epoch 2: val_acc improved from 0.87983 to 0.89933, saving model to /content/drive/MyDrive/experiments/fashion32/evo/fold5/cnn_saved_model/weights-improvement--02-0.27.hdf5
1688/1688 [==============================] - 76s 45ms/step - loss: 0.3063 - acc: 0.8887 - val_loss: 0.2730 - val_acc: 0.8993
Epoch 3/50
1687/1688 [============================>.] - ETA: 0s - loss: 0.2660 - acc: 0.9020
Epoch 3: val_acc improved from 0.89933 to 0.91050, saving model to /content/drive/MyDrive/experiments/fashion32/evo/fold5/cnn_saved_model/wei

# Bayes

In [ ]:
tf.random.set_seed(1234) #fashion 32 bayes fold 1
input_shape = (28, 28, 1)
conv1 = 124
conv2 = 66
cov3 = 70
l_rate = 5
kernel1 = 3
kernel2=3
kernel3 = 15
drop1 = 3
drop2 = 6


# optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/fashion32/bayes/fold1/Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/fashion32/bayes/fold1/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)

history = model.fit(x_train, y_train,  #fashion 32 bayes fold 1
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
1688/1688 [==============================] - ETA: 0s - loss: 0.4544 - acc: 0.8342
Epoch 1: val_acc improved from -inf to 0.88600, saving model to /content/drive/MyDrive/experiments/fashion32/bayes/fold1/cnn_saved_model/weights-improvement--01-0.31.hdf5
1688/1688 [==============================] - 71s 41ms/step - loss: 0.4544 - acc: 0.8342 - val_loss: 0.3067 - val_acc: 0.8860
Epoch 2/50
1687/1688 [============================>.] - ETA: 0s - loss: 0.3001 - acc: 0.8909
Epoch 2: val_acc improved from 0.88600 to 0.90517, saving model to /content/drive/MyDrive/experiments/fashion32/bayes/fold1/cnn_saved_model/weights-improvement--02-0.26.hdf5
1688/1688 [==============================] - 69s 41ms/step - loss: 0.3002 - acc: 0.8909 - val_loss: 0.2556 - val_acc: 0.9052
Epoch 3/50
1688/1688 [==============================] - ETA: 0s - loss: 0.2590 - acc: 0.9067
Epoch 3: val_acc improved from 0.90517 to 0.91583, saving model to /content/drive/MyDrive/experiments/fashion32/bayes/fold1/cn

In [ ]:
tf.random.set_seed(1234) #fashion 32 bayes fold 2
input_shape = (28, 28, 1)
conv1 = 136
conv2 = 100
cov3 = 80
l_rate = 5
kernel1 = 3
kernel2=3
kernel3 = 15
drop1 = 3
drop2 = 6


# optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/fashion32/bayes/fold2/Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/fashion32/bayes/fold2/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)

history = model.fit(x_train, y_train,  #fashion 32 bayes fold 2
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
1688/1688 [==============================] - ETA: 0s - loss: 0.4052 - acc: 0.8526
Epoch 1: val_acc improved from -inf to 0.89967, saving model to /content/drive/MyDrive/experiments/fashion32/bayes/fold2/cnn_saved_model/weights-improvement--01-0.27.hdf5
1688/1688 [==============================] - 68s 39ms/step - loss: 0.4052 - acc: 0.8526 - val_loss: 0.2716 - val_acc: 0.8997
Epoch 2/50
1688/1688 [==============================] - ETA: 0s - loss: 0.2617 - acc: 0.9040
Epoch 2: val_acc improved from 0.89967 to 0.91767, saving model to /content/drive/MyDrive/experiments/fashion32/bayes/fold2/cnn_saved_model/weights-improvement--02-0.23.hdf5
1688/1688 [==============================] - 66s 39ms/step - loss: 0.2617 - acc: 0.9040 - val_loss: 0.2275 - val_acc: 0.9177
Epoch 3/50
1688/1688 [==============================] - ETA: 0s - loss: 0.2191 - acc: 0.9199
Epoch 3: val_acc improved from 0.91767 to 0.92267, saving model to /content/drive/MyDrive/experiments/fashion32/bayes/fold2/cn

In [ ]:
tf.random.set_seed(1234) #fashion 32 bayes fold 3
input_shape = (28, 28, 1)
conv1 = 140
conv2 = 100
cov3 = 32
l_rate = 5
kernel1 = 3
kernel2=3
kernel3 = 15
drop1 = 3
drop2 = 6


# optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/fashion32/bayes/fold3/Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/fashion32/bayes/fold3/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)

history = model.fit(x_train, y_train,  #fashion 32 bayes fold 3
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
1688/1688 [==============================] - ETA: 0s - loss: 0.4334 - acc: 0.8458
Epoch 1: val_acc improved from -inf to 0.89467, saving model to /content/drive/MyDrive/experiments/fashion32/bayes/fold3/cnn_saved_model/weights-improvement--01-0.29.hdf5
1688/1688 [==============================] - 71s 41ms/step - loss: 0.4334 - acc: 0.8458 - val_loss: 0.2892 - val_acc: 0.8947
Epoch 2/50
1688/1688 [==============================] - ETA: 0s - loss: 0.2845 - acc: 0.8971
Epoch 2: val_acc improved from 0.89467 to 0.91067, saving model to /content/drive/MyDrive/experiments/fashion32/bayes/fold3/cnn_saved_model/weights-improvement--02-0.24.hdf5
1688/1688 [==============================] - 69s 41ms/step - loss: 0.2845 - acc: 0.8971 - val_loss: 0.2396 - val_acc: 0.9107
Epoch 3/50
1688/1688 [==============================] - ETA: 0s - loss: 0.2418 - acc: 0.9127
Epoch 3: val_acc improved from 0.91067 to 0.91967, saving model to /content/drive/MyDrive/experiments/fashion32/bayes/fold3/cn

In [ ]:
tf.random.set_seed(1234) #fashion 32 bayes fold 4
input_shape = (28, 28, 1)
conv1 = 40
conv2 = 100
cov3 = 80
l_rate = 5
kernel1 = 3
kernel2=3
kernel3 = 15
drop1 = 3
drop2 = 4


# optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/fashion32/bayes/fold4/Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/fashion32/bayes/fold4/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)

history = model.fit(x_train, y_train,  #fashion 32 bayes fold 4
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
1688/1688 [==============================] - ETA: 0s - loss: 0.3921 - acc: 0.8591
Epoch 1: val_acc improved from -inf to 0.89733, saving model to /content/drive/MyDrive/experiments/fashion32/bayes/fold4/cnn_saved_model/weights-improvement--01-0.28.hdf5
1688/1688 [==============================] - 70s 40ms/step - loss: 0.3921 - acc: 0.8591 - val_loss: 0.2775 - val_acc: 0.8973
Epoch 2/50
1688/1688 [==============================] - ETA: 0s - loss: 0.2503 - acc: 0.9084
Epoch 2: val_acc improved from 0.89733 to 0.91683, saving model to /content/drive/MyDrive/experiments/fashion32/bayes/fold4/cnn_saved_model/weights-improvement--02-0.23.hdf5
1688/1688 [==============================] - 65s 39ms/step - loss: 0.2503 - acc: 0.9084 - val_loss: 0.2284 - val_acc: 0.9168
Epoch 3/50
1688/1688 [==============================] - ETA: 0s - loss: 0.2054 - acc: 0.9248
Epoch 3: val_acc improved from 0.91683 to 0.92450, saving model to /content/drive/MyDrive/experiments/fashion32/bayes/fold4/cn

In [ ]:
tf.random.set_seed(1234) #fashion 32 bayes fold 5
input_shape = (28, 28, 1)
conv1 = 140
conv2 = 82
cov3 = 80
l_rate = 5
kernel1 = 3
kernel2=3
kernel3 = 15
drop1 = 3
drop2 = 6


# optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/fashion32/bayes/fold5/Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/fashion32/bayes/fold5/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)

history = model.fit(x_train, y_train,  #fashion 32 bayes fold 5
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
1688/1688 [==============================] - ETA: 0s - loss: 0.4075 - acc: 0.8526
Epoch 1: val_acc improved from -inf to 0.89817, saving model to /content/drive/MyDrive/experiments/fashion32/bayes/fold5/cnn_saved_model/weights-improvement--01-0.28.hdf5
1688/1688 [==============================] - 65s 38ms/step - loss: 0.4075 - acc: 0.8526 - val_loss: 0.2778 - val_acc: 0.8982
Epoch 2/50
1687/1688 [============================>.] - ETA: 0s - loss: 0.2658 - acc: 0.9029
Epoch 2: val_acc improved from 0.89817 to 0.91650, saving model to /content/drive/MyDrive/experiments/fashion32/bayes/fold5/cnn_saved_model/weights-improvement--02-0.23.hdf5
1688/1688 [==============================] - 63s 37ms/step - loss: 0.2658 - acc: 0.9029 - val_loss: 0.2297 - val_acc: 0.9165
Epoch 3/50
1688/1688 [==============================] - ETA: 0s - loss: 0.2252 - acc: 0.9180
Epoch 3: val_acc improved from 0.91650 to 0.92383, saving model to /content/drive/MyDrive/experiments/fashion32/bayes/fold5/cn

# Random

In [ ]:
tf.random.set_seed(1234) #fashion 32 random fold 1
input_shape = (28, 28, 1)
conv1 = 100
conv2 = 79
cov3 = 58
l_rate = 6
kernel1 = 3
kernel2=3
kernel3 = 13
drop1 = 3
drop2 = 4


optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
# optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/fashion32/random/fold1/Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/fashion32/random/fold1/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)

history = model.fit(x_train, y_train,  #fashion 32 random fold 1
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
1688/1688 [==============================] - ETA: 0s - loss: 0.3901 - acc: 0.8592
Epoch 1: val_acc improved from -inf to 0.89817, saving model to /content/drive/MyDrive/experiments/fashion32/random/fold1/cnn_saved_model/weights-improvement--01-0.28.hdf5
1688/1688 [==============================] - 74s 42ms/step - loss: 0.3901 - acc: 0.8592 - val_loss: 0.2788 - val_acc: 0.8982
Epoch 2/50
1688/1688 [==============================] - ETA: 0s - loss: 0.2528 - acc: 0.9081
Epoch 2: val_acc improved from 0.89817 to 0.91350, saving model to /content/drive/MyDrive/experiments/fashion32/random/fold1/cnn_saved_model/weights-improvement--02-0.23.hdf5
1688/1688 [==============================] - 72s 43ms/step - loss: 0.2528 - acc: 0.9081 - val_loss: 0.2309 - val_acc: 0.9135
Epoch 3/50
1687/1688 [============================>.] - ETA: 0s - loss: 0.2139 - acc: 0.9232
Epoch 3: val_acc improved from 0.91350 to 0.92183, saving model to /content/drive/MyDrive/experiments/fashion32/random/fold1

In [ ]:
tf.random.set_seed(1234) #fashion 32 random fold 2
input_shape = (28, 28, 1)
conv1 = 134
conv2 = 96
cov3 = 49
l_rate = 6
kernel1 = 3
kernel2=5
kernel3 = 11
drop1 = 6
drop2 = 4


optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
# optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/fashion32/random/fold2/Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/fashion32/random/fold2/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)

history = model.fit(x_train, y_train,  #fashion 32 random fold 2
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
1688/1688 [==============================] - ETA: 0s - loss: 0.4079 - acc: 0.8518
Epoch 1: val_acc improved from -inf to 0.89367, saving model to /content/drive/MyDrive/experiments/fashion32/random/fold2/cnn_saved_model/weights-improvement--01-0.28.hdf5
1688/1688 [==============================] - 78s 44ms/step - loss: 0.4079 - acc: 0.8518 - val_loss: 0.2826 - val_acc: 0.8937
Epoch 2/50
1688/1688 [==============================] - ETA: 0s - loss: 0.2751 - acc: 0.8997
Epoch 2: val_acc improved from 0.89367 to 0.91300, saving model to /content/drive/MyDrive/experiments/fashion32/random/fold2/cnn_saved_model/weights-improvement--02-0.24.hdf5
1688/1688 [==============================] - 75s 44ms/step - loss: 0.2751 - acc: 0.8997 - val_loss: 0.2382 - val_acc: 0.9130
Epoch 3/50
1687/1688 [============================>.] - ETA: 0s - loss: 0.2394 - acc: 0.9120
Epoch 3: val_acc improved from 0.91300 to 0.91450, saving model to /content/drive/MyDrive/experiments/fashion32/random/fold2

In [ ]:
tf.random.set_seed(1234) #fashion 32 random fold 3
input_shape = (28, 28, 1)
conv1 = 71
conv2 = 90
cov3 = 66
l_rate = 5
kernel1 = 5
kernel2=7
kernel3 = 13
drop1 = 4
drop2 = 3


optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
# optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/fashion32/random/fold3/Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/fashion32/random/fold3/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)

history = model.fit(x_train, y_train,  #fashion 32 random fold 3
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
1688/1688 [==============================] - ETA: 0s - loss: 0.3922 - acc: 0.8592
Epoch 1: val_acc improved from -inf to 0.89417, saving model to /content/drive/MyDrive/experiments/fashion32/random/fold3/cnn_saved_model/weights-improvement--01-0.29.hdf5
1688/1688 [==============================] - 80s 46ms/step - loss: 0.3922 - acc: 0.8592 - val_loss: 0.2939 - val_acc: 0.8942
Epoch 2/50
1687/1688 [============================>.] - ETA: 0s - loss: 0.2624 - acc: 0.9034
Epoch 2: val_acc improved from 0.89417 to 0.91083, saving model to /content/drive/MyDrive/experiments/fashion32/random/fold3/cnn_saved_model/weights-improvement--02-0.25.hdf5
1688/1688 [==============================] - 76s 45ms/step - loss: 0.2624 - acc: 0.9034 - val_loss: 0.2487 - val_acc: 0.9108
Epoch 3/50
1687/1688 [============================>.] - ETA: 0s - loss: 0.2222 - acc: 0.9184
Epoch 3: val_acc improved from 0.91083 to 0.91583, saving model to /content/drive/MyDrive/experiments/fashion32/random/fold3

In [ ]:
tf.random.set_seed(1234) #fashion 32 random fold 4
input_shape = (28, 28, 1)
conv1 = 128
conv2 = 73
cov3 = 47
l_rate = 7
kernel1 = 5
kernel2=3
kernel3 = 15
drop1 = 4
drop2 = 5


optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
# optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/fashion32/random/fold4/Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/fashion32/random/fold4/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)

history = model.fit(x_train, y_train,  #fashion 32 random fold 4
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
1688/1688 [==============================] - ETA: 0s - loss: 0.4092 - acc: 0.8535
Epoch 1: val_acc improved from -inf to 0.89467, saving model to /content/drive/MyDrive/experiments/fashion32/random/fold4/cnn_saved_model/weights-improvement--01-0.30.hdf5
1688/1688 [==============================] - 83s 47ms/step - loss: 0.4092 - acc: 0.8535 - val_loss: 0.3013 - val_acc: 0.8947
Epoch 2/50
1687/1688 [============================>.] - ETA: 0s - loss: 0.2778 - acc: 0.8973
Epoch 2: val_acc improved from 0.89467 to 0.90633, saving model to /content/drive/MyDrive/experiments/fashion32/random/fold4/cnn_saved_model/weights-improvement--02-0.26.hdf5
1688/1688 [==============================] - 79s 47ms/step - loss: 0.2778 - acc: 0.8973 - val_loss: 0.2598 - val_acc: 0.9063
Epoch 3/50
1688/1688 [==============================] - ETA: 0s - loss: 0.2398 - acc: 0.9126
Epoch 3: val_acc improved from 0.90633 to 0.91567, saving model to /content/drive/MyDrive/experiments/fashion32/random/fold4

In [ ]:
tf.random.set_seed(1234) #fashion 32 random fold 5
input_shape = (28, 28, 1)
conv1 = 123
conv2 = 82
cov3 = 65
l_rate = 7
kernel1 = 3
kernel2=3
kernel3 = 11
drop1 = 4
drop2 = 5


# optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/fashion32/random/fold5/Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/fashion32/random/fold5/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)

history = model.fit(x_train, y_train,  #fashion 32 random fold 5
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
1688/1688 [==============================] - ETA: 0s - loss: 0.4028 - acc: 0.8538
Epoch 1: val_acc improved from -inf to 0.90333, saving model to /content/drive/MyDrive/experiments/fashion32/random/fold5/cnn_saved_model/weights-improvement--01-0.27.hdf5
1688/1688 [==============================] - 68s 39ms/step - loss: 0.4028 - acc: 0.8538 - val_loss: 0.2686 - val_acc: 0.9033
Epoch 2/50
1688/1688 [==============================] - ETA: 0s - loss: 0.2692 - acc: 0.9021
Epoch 2: val_acc improved from 0.90333 to 0.91300, saving model to /content/drive/MyDrive/experiments/fashion32/random/fold5/cnn_saved_model/weights-improvement--02-0.23.hdf5
1688/1688 [==============================] - 64s 38ms/step - loss: 0.2692 - acc: 0.9021 - val_loss: 0.2330 - val_acc: 0.9130
Epoch 3/50
1687/1688 [============================>.] - ETA: 0s - loss: 0.2319 - acc: 0.9165
Epoch 3: val_acc improved from 0.91300 to 0.92033, saving model to /content/drive/MyDrive/experiments/fashion32/random/fold5

# Hyperband

In [ ]:
tf.random.set_seed(1234) #fashion 32 hyperband fold 1
input_shape = (28, 28, 1)
conv1 = 99
conv2 = 87
cov3 = 64
l_rate = 5
kernel1 = 5
kernel2=5
kernel3 = 15
drop1 = 4
drop2 = 4


optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
# optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/fashion32/hyperband/fold1/Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/fashion32/hyperband/fold1/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)

history = model.fit(x_train, y_train,  #fashion 32 hyperband fold 1
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
1688/1688 [==============================] - ETA: 0s - loss: 0.3961 - acc: 0.8584
Epoch 1: val_acc improved from -inf to 0.89367, saving model to /content/drive/MyDrive/experiments/fashion32/hyperband/fold1/cnn_saved_model/weights-improvement--01-0.30.hdf5
1688/1688 [==============================] - 86s 50ms/step - loss: 0.3961 - acc: 0.8584 - val_loss: 0.2953 - val_acc: 0.8937
Epoch 2/50
1687/1688 [============================>.] - ETA: 0s - loss: 0.2634 - acc: 0.9040
Epoch 2: val_acc improved from 0.89367 to 0.90967, saving model to /content/drive/MyDrive/experiments/fashion32/hyperband/fold1/cnn_saved_model/weights-improvement--02-0.25.hdf5
1688/1688 [==============================] - 83s 49ms/step - loss: 0.2634 - acc: 0.9040 - val_loss: 0.2502 - val_acc: 0.9097
Epoch 3/50
1688/1688 [==============================] - ETA: 0s - loss: 0.2250 - acc: 0.9166
Epoch 3: val_acc improved from 0.90967 to 0.91733, saving model to /content/drive/MyDrive/experiments/fashion32/hyperb

In [ ]:
tf.random.set_seed(1234) #fashion 32 hyperband fold 2
input_shape = (28, 28, 1)
conv1 = 77
conv2 = 94
cov3 = 47
l_rate = 5
kernel1 = 3
kernel2=7
kernel3 = 13
drop1 = 4
drop2 = 6


# optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/fashion32/hyperband/fold2/Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/fashion32/hyperband/fold2/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)

history = model.fit(x_train, y_train,  #fashion 32 hyperband fold 2
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
1688/1688 [==============================] - ETA: 0s - loss: 0.4348 - acc: 0.8427
Epoch 1: val_acc improved from -inf to 0.89217, saving model to /content/drive/MyDrive/experiments/fashion32/hyperband/fold2/cnn_saved_model/weights-improvement--01-0.29.hdf5
1688/1688 [==============================] - 66s 38ms/step - loss: 0.4348 - acc: 0.8427 - val_loss: 0.2922 - val_acc: 0.8922
Epoch 2/50
1687/1688 [============================>.] - ETA: 0s - loss: 0.2852 - acc: 0.8954
Epoch 2: val_acc improved from 0.89217 to 0.91167, saving model to /content/drive/MyDrive/experiments/fashion32/hyperband/fold2/cnn_saved_model/weights-improvement--02-0.24.hdf5
1688/1688 [==============================] - 64s 38ms/step - loss: 0.2852 - acc: 0.8954 - val_loss: 0.2433 - val_acc: 0.9117
Epoch 3/50
1688/1688 [==============================] - ETA: 0s - loss: 0.2454 - acc: 0.9093
Epoch 3: val_acc improved from 0.91167 to 0.91550, saving model to /content/drive/MyDrive/experiments/fashion32/hyperb

In [ ]:
tf.random.set_seed(1234) #fashion 32 hyperband fold 3
input_shape = (28, 28, 1)
conv1 = 85
conv2 = 47
cov3 = 40
l_rate = 5
kernel1 = 3
kernel2=5
kernel3 = 5
drop1 = 4
drop2 = 5


# optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/fashion32/hyperband/fold3/Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/fashion32/hyperband/fold3/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)

history = model.fit(x_train, y_train,  #fashion 32 hyperband fold 3
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
1688/1688 [==============================] - ETA: 0s - loss: 0.4733 - acc: 0.8281
Epoch 1: val_acc improved from -inf to 0.88567, saving model to /content/drive/MyDrive/experiments/fashion32/hyperband/fold3/cnn_saved_model/weights-improvement--01-0.31.hdf5
1688/1688 [==============================] - 69s 39ms/step - loss: 0.4733 - acc: 0.8281 - val_loss: 0.3100 - val_acc: 0.8857
Epoch 2/50
1687/1688 [============================>.] - ETA: 0s - loss: 0.3090 - acc: 0.8899
Epoch 2: val_acc improved from 0.88567 to 0.90400, saving model to /content/drive/MyDrive/experiments/fashion32/hyperband/fold3/cnn_saved_model/weights-improvement--02-0.27.hdf5
1688/1688 [==============================] - 66s 39ms/step - loss: 0.3090 - acc: 0.8899 - val_loss: 0.2657 - val_acc: 0.9040
Epoch 3/50
1688/1688 [==============================] - ETA: 0s - loss: 0.2708 - acc: 0.9008
Epoch 3: val_acc improved from 0.90400 to 0.91300, saving model to /content/drive/MyDrive/experiments/fashion32/hyperb

In [ ]:
tf.random.set_seed(1234) #fashion 32 hyperband fold 4
input_shape = (28, 28, 1)
conv1 = 75
conv2 = 98
cov3 = 40
l_rate = 11
kernel1 = 3
kernel2=3
kernel3 = 15
drop1 = 4
drop2 = 5


# optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/fashion32/hyperband/fold4/Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/fashion32/hyperband/fold4/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)

history = model.fit(x_train, y_train,  #fashion 32 hyperband fold 4
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
1688/1688 [==============================] - ETA: 0s - loss: 0.4005 - acc: 0.8551
Epoch 1: val_acc improved from -inf to 0.89850, saving model to /content/drive/MyDrive/experiments/fashion32/hyperband/fold4/cnn_saved_model/weights-improvement--01-0.27.hdf5
1688/1688 [==============================] - 76s 43ms/step - loss: 0.4005 - acc: 0.8551 - val_loss: 0.2707 - val_acc: 0.8985
Epoch 2/50
1688/1688 [==============================] - ETA: 0s - loss: 0.2735 - acc: 0.8996
Epoch 2: val_acc improved from 0.89850 to 0.91450, saving model to /content/drive/MyDrive/experiments/fashion32/hyperband/fold4/cnn_saved_model/weights-improvement--02-0.23.hdf5
1688/1688 [==============================] - 70s 41ms/step - loss: 0.2735 - acc: 0.8996 - val_loss: 0.2295 - val_acc: 0.9145
Epoch 3/50
1687/1688 [============================>.] - ETA: 0s - loss: 0.2383 - acc: 0.9121
Epoch 3: val_acc improved from 0.91450 to 0.91817, saving model to /content/drive/MyDrive/experiments/fashion32/hyperb

In [ ]:
tf.random.set_seed(1234) #fashion 32 hyperband fold 5
input_shape = (28, 28, 1)
conv1 = 73
conv2 = 97
cov3 = 50
l_rate = 5
kernel1 = 3
kernel2=7
kernel3 = 13
drop1 = 5
drop2 = 4


optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
# optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/fashion32/hyperband/fold5/Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/fashion32/hyperband/fold5/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)

history = model.fit(x_train, y_train,  #fashion 32 hyperband fold 5
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
1688/1688 [==============================] - ETA: 0s - loss: 0.4098 - acc: 0.8521
Epoch 1: val_acc improved from -inf to 0.89450, saving model to /content/drive/MyDrive/experiments/fashion32/hyperband/fold5/cnn_saved_model/weights-improvement--01-0.30.hdf5
1688/1688 [==============================] - 83s 47ms/step - loss: 0.4098 - acc: 0.8521 - val_loss: 0.2955 - val_acc: 0.8945
Epoch 2/50
1687/1688 [============================>.] - ETA: 0s - loss: 0.2722 - acc: 0.8998
Epoch 2: val_acc improved from 0.89450 to 0.91067, saving model to /content/drive/MyDrive/experiments/fashion32/hyperband/fold5/cnn_saved_model/weights-improvement--02-0.25.hdf5
1688/1688 [==============================] - 80s 48ms/step - loss: 0.2723 - acc: 0.8998 - val_loss: 0.2500 - val_acc: 0.9107
Epoch 3/50
1687/1688 [============================>.] - ETA: 0s - loss: 0.2329 - acc: 0.9149
Epoch 3: val_acc improved from 0.91067 to 0.91317, saving model to /content/drive/MyDrive/experiments/fashion32/hyperb

# SH

In [ ]:
tf.random.set_seed(1234) #fashion 32 sh fold 1
input_shape = (28, 28, 1)
conv1 = 98
conv2 = 93
cov3 = 64
l_rate = 5
kernel1 = 3
kernel2=5
kernel3 = 13
drop1 = 5
drop2 = 3


optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
# optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/fashion32/sh/fold1/Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/fashion32/sh/fold1/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)

history = model.fit(x_train, y_train,  #fashion 32 sh fold 1
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50


1688/1688 [==============================] - ETA: 0s - loss: 0.4241 - acc: 0.8468
Epoch 1: val_acc improved from -inf to 0.88667, saving model to /content/drive/MyDrive/experiments/fashion32/sh/fold1/cnn_saved_model/weights-improvement--01-0.31.hdf5
1688/1688 [==============================] - 94s 49ms/step - loss: 0.4241 - acc: 0.8468 - val_loss: 0.3121 - val_acc: 0.8867
Epoch 2/50
1687/1688 [============================>.] - ETA: 0s - loss: 0.2823 - acc: 0.8973
Epoch 2: val_acc improved from 0.88667 to 0.91017, saving model to /content/drive/MyDrive/experiments/fashion32/sh/fold1/cnn_saved_model/weights-improvement--02-0.25.hdf5
1688/1688 [==============================] - 84s 50ms/step - loss: 0.2822 - acc: 0.8973 - val_loss: 0.2470 - val_acc: 0.9102
Epoch 3/50
1687/1688 [============================>.] - ETA: 0s - loss: 0.2432 - acc: 0.9112
Epoch 3: val_acc improved from 0.91017 to 0.91700, saving model to /content/drive/MyDrive/experiments/fashion32/sh/fold1/cnn_saved_model/weight

In [ ]:
tf.random.set_seed(1234) #fashion 32 sh fold 2
input_shape = (28, 28, 1)
conv1 = 136
conv2 = 96
cov3 = 69
l_rate = 5
kernel1 = 5
kernel2=3
kernel3 = 13
drop1 = 3
drop2 = 3


optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
# optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/fashion32/sh/fold2/Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/fashion32/sh/fold2/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)

history = model.fit(x_train, y_train,  #fashion 32 sh fold 2
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50


1688/1688 [==============================] - ETA: 0s - loss: 0.4221 - acc: 0.8472
Epoch 1: val_acc improved from -inf to 0.87883, saving model to /content/drive/MyDrive/experiments/fashion32/sh/fold2/cnn_saved_model/weights-improvement--01-0.33.hdf5
1688/1688 [==============================] - 82s 46ms/step - loss: 0.4221 - acc: 0.8472 - val_loss: 0.3289 - val_acc: 0.8788
Epoch 2/50
1688/1688 [==============================] - ETA: 0s - loss: 0.2805 - acc: 0.8980
Epoch 2: val_acc improved from 0.87883 to 0.90950, saving model to /content/drive/MyDrive/experiments/fashion32/sh/fold2/cnn_saved_model/weights-improvement--02-0.26.hdf5
1688/1688 [==============================] - 77s 46ms/step - loss: 0.2805 - acc: 0.8980 - val_loss: 0.2623 - val_acc: 0.9095
Epoch 3/50
1688/1688 [==============================] - ETA: 0s - loss: 0.2421 - acc: 0.9117
Epoch 3: val_acc improved from 0.90950 to 0.91550, saving model to /content/drive/MyDrive/experiments/fashion32/sh/fold2/cnn_saved_model/weight

In [ ]:
tf.random.set_seed(1234) #fashion 32 sh fold 3
input_shape = (28, 28, 1)
conv1 = 53
conv2 = 66
cov3 = 64
l_rate = 6
kernel1 = 3
kernel2=3
kernel3 = 7
drop1 = 5
drop2 = 3


optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
# optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/fashion32/sh/fold3/Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/fashion32/sh/fold3/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)

history = model.fit(x_train, y_train,  #fashion 32 sh fold 3
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
1688/1688 [==============================] - ETA: 0s - loss: 0.4121 - acc: 0.8530
Epoch 1: val_acc improved from -inf to 0.90067, saving model to /content/drive/MyDrive/experiments/fashion32/sh/fold3/cnn_saved_model/weights-improvement--01-0.28.hdf5
1688/1688 [==============================] - 69s 40ms/step - loss: 0.4121 - acc: 0.8530 - val_loss: 0.2774 - val_acc: 0.9007
Epoch 2/50
1687/1688 [============================>.] - ETA: 0s - loss: 0.2708 - acc: 0.9028
Epoch 2: val_acc improved from 0.90067 to 0.91267, saving model to /content/drive/MyDrive/experiments/fashion32/sh/fold3/cnn_saved_model/weights-improvement--02-0.24.hdf5
1688/1688 [==============================] - 63s 37ms/step - loss: 0.2708 - acc: 0.9028 - val_loss: 0.2377 - val_acc: 0.9127
Epoch 3/50
1687/1688 [============================>.] - ETA: 0s - loss: 0.2339 - acc: 0.9151
Epoch 3: val_acc improved from 0.91267 to 0.91917, saving model to /content/drive/MyDrive/experiments/fashion32/sh/fold3/cnn_saved_m

In [ ]:
tf.random.set_seed(1234) #fashion 32 sh fold 4
input_shape = (28, 28, 1)
conv1 = 45
conv2 = 100
cov3 = 33
l_rate = 7
kernel1 = 3
kernel2=5
kernel3 = 13
drop1 = 3
drop2 = 5


optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
# optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/fashion32/sh/fold4/Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/fashion32/sh/fold4/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)

history = model.fit(x_train, y_train,  #fashion 32 sh fold 4
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
1688/1688 [==============================] - ETA: 0s - loss: 0.4052 - acc: 0.8533
Epoch 1: val_acc improved from -inf to 0.90283, saving model to /content/drive/MyDrive/experiments/fashion32/sh/fold4/cnn_saved_model/weights-improvement--01-0.27.hdf5
1688/1688 [==============================] - 75s 43ms/step - loss: 0.4052 - acc: 0.8533 - val_loss: 0.2692 - val_acc: 0.9028
Epoch 2/50
1688/1688 [==============================] - ETA: 0s - loss: 0.2649 - acc: 0.9030
Epoch 2: val_acc improved from 0.90283 to 0.91700, saving model to /content/drive/MyDrive/experiments/fashion32/sh/fold4/cnn_saved_model/weights-improvement--02-0.23.hdf5
1688/1688 [==============================] - 71s 42ms/step - loss: 0.2649 - acc: 0.9030 - val_loss: 0.2319 - val_acc: 0.9170
Epoch 3/50
1688/1688 [==============================] - ETA: 0s - loss: 0.2239 - acc: 0.9186
Epoch 3: val_acc improved from 0.91700 to 0.92317, saving model to /content/drive/MyDrive/experiments/fashion32/sh/fold4/cnn_saved_m

In [ ]:
tf.random.set_seed(1234) #fashion 32 sh fold 5
input_shape = (28, 28, 1)
conv1 = 105
conv2 = 78
cov3 = 63
l_rate = 5
kernel1 = 3
kernel2=3
kernel3 = 13
drop1 = 3
drop2 = 5


optimizer = tf.keras.optimizers.Nadam(learning_rate=l_rate / 10000)
# optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate / 10000)
model = tf.keras.models.Sequential([
    RandomInvert(),
    tf.keras.layers.Conv2D(conv1, (kernel1, kernel1), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(conv2, (kernel2, kernel2), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(drop1 / 10),
    tf.keras.layers.Conv2D(cov3, (kernel3, kernel3), padding='same', activation='relu'),
    tf.keras.layers.Dropout(drop2/ 10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')])
tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/experiments/fashion32/sh/fold5/Tensorboard', update_freq='batch',)
# keras.callbacks.TensorBoard(".//drive//MyDrive//kerastuner/random_fold2")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, baseline= 0.5)
filepath = '/content/drive/MyDrive/experiments/fashion32/sh/fold5/cnn_saved_model/weights-improvement--{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc',verbose=1,  save_best_only=True, mode='max')
num_ep = LambdaCallback(
    on_epoch_end=lambda epoch,logs: epoch_track.increase())
callbacks_list = [num_ep, checkpoint, tensorboard]



model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'],  run_eagerly=True)

history = model.fit(x_train, y_train,  #fashion 32 sh fold 5
                    batch_size=32,
                    epochs=50,
                    validation_split=0.1,
                    callbacks=callbacks_list,
                    verbose=1
                    )
end = time.time()
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(end-start)))
eval_metrics = model.evaluate(x_test, y_test)


Epoch 1/50
1688/1688 [==============================] - ETA: 0s - loss: 0.4035 - acc: 0.8560
Epoch 1: val_acc improved from -inf to 0.89350, saving model to /content/drive/MyDrive/experiments/fashion32/sh/fold5/cnn_saved_model/weights-improvement--01-0.29.hdf5
1688/1688 [==============================] - 77s 44ms/step - loss: 0.4035 - acc: 0.8560 - val_loss: 0.2939 - val_acc: 0.8935
Epoch 2/50
1687/1688 [============================>.] - ETA: 0s - loss: 0.2627 - acc: 0.9057
Epoch 2: val_acc improved from 0.89350 to 0.91300, saving model to /content/drive/MyDrive/experiments/fashion32/sh/fold5/cnn_saved_model/weights-improvement--02-0.24.hdf5
1688/1688 [==============================] - 74s 44ms/step - loss: 0.2628 - acc: 0.9057 - val_loss: 0.2354 - val_acc: 0.9130
Epoch 3/50
1688/1688 [==============================] - ETA: 0s - loss: 0.2213 - acc: 0.9187
Epoch 3: val_acc improved from 0.91300 to 0.91983, saving model to /content/drive/MyDrive/experiments/fashion32/sh/fold5/cnn_saved_m